# Ecobici

## Import

In [3]:
import sys
sys.path.append('/Users/efraflores/Desktop/hub/diplo/venv/lib/python3.9/site-packages')

In [4]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/data/practice'

In [5]:
import os
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format

df = pd.read_csv(os.path.join(BASE_DIR,'2019-05.csv'))
df.sample()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo
310489,M,38,12231,334,14/05/2019,15:07:59,382,14/05/2019,15:14:13


In [6]:
geo = pd.read_csv(os.path.join(BASE_DIR,'estaciones-de-ecobici.csv'))
geo.sample()

,ID,Nombre,Dirección,Número,Código postal,districtCode,Colonia,altitude,nearbyStations/0,location/lat,location/lon,Tipo de estación,nearbyStations/1,nearbyStations/2,nearbyStations/3,nearbyStations/4,nearbyStations/5,punto_geo
33,150,150 CAMPECHE-MEDELLÍN,150 - Campeche-Medellín,S/N,NaN,1,Ampliación Granada,NaN,146,19.410,-99.164,BIKE,151.000,152.000,NaN,NaN,NaN,"19.409965,-99.164292"


## Functions

In [7]:
import time
import numpy as np
from IPython.lib.display import Audio

#Start a "stopwatch"
start = time.time()
def time_exp(x):
    #Just print how many minutes and seconds have passed
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tone(a=1000, b=700, play_time_seconds=1, framerate=4410):
    #Make a sound! Useful while training models
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t)+np.sin(b*t), rate=framerate, autoplay=True)

In [9]:
#Uncomment the next line if you don't have it installed yet, 
!pip install mpu

#it has the function to calculate the distance between two coordinates
import mpu

def transf(data,tipos):
    df = data.copy()
    for tipo in tipos:
        #Merge de date and hour columns, and set is a datetime column
        df[f'Fecha_{tipo}'] = pd.to_datetime(df[f'Fecha_{tipo}']+','+df[f'Hora_{tipo}'],
                                              format='%d/%m/%Y,%H:%M:%S')
        #Get the day of the week name, just the first 3 characters
        df[f'DoW_{tipo}'] = df[f'Fecha_{tipo}'].dt.day_name().str[:3]
        #Get the time as hour-decimal. Ex: 15:30 --> 15.5
        df[f'HRmin_{tipo}'] = (df[f'Fecha_{tipo}'].dt.hour+
                               df[f'Fecha_{tipo}'].dt.minute/60+
                               df[f'Fecha_{tipo}'].dt.second/60**2)
        #Make equally-distributed bins and save the bins
        df[f'HRmin_{tipo}_cut'],qbins = pd.qcut(df[f'HRmin_{tipo}'].apply(lambda x:round(x,3)),
                                                q=7,duplicates='drop',retbins=True)
        df[f'HRmin_{tipo}_cut'] = df[f'HRmin_{tipo}_cut'].astype(str)
        #Get the geo-points from the origin and destination stations
        df = df.merge(geo.set_index('ID')['punto_geo'].str.split(',').to_frame().reset_index(),
                      left_on=f'Ciclo_Estacion_{tipo}',right_on='ID'
                     ).drop(columns='ID').rename(columns={'punto_geo':f'geo_{tipo.lower()}'})
    #Calculate the ridetime in hours
    df['Ridetime'] = df['HRmin_Arribo'] - df['HRmin_Retiro']
    #There were few trips at midnight (why?)
    df = df[df['Ridetime']>0].copy()
    #Distance in km between the destination vs the origin station 
    df['Distance'] = [mpu.haversine_distance(tuple([float(a) for a in x]),tuple([float(a) for a in y]))
                      for x,y in zip(df['geo_retiro'],df['geo_arribo'])]
    #Velocity of every ride in km/hr
    df['Velocity'] = df['Distance'] / df['Ridetime']
    #Get rid of unnecesary columns
    df.drop(columns=['Hora_Retiro','Hora_Arribo','geo_retiro','geo_arribo'],inplace=True)
    return df,qbins

  Using cached mpu-0.23.1-py3-none-any.whl (69 kB)


In [10]:
def iqr(data,x,p=0.3):
    var = data[x].copy()
    q1 = var.quantile(p/2)
    q3 = var.quantile(1-p/2)
    iqr = q3 - q1
    return data[(var.isnull()) | ((var >= q1 - 1.5*iqr) & (var <= q3 + 1.5*iqr))].copy()

In [11]:
import numpy as np

def perc70(x):
    return np.percentile(x,70)

In [12]:
def multishift(data,id_cols,date_col,shifts,**pivot_args):
    df = data.copy()
    #Make sure the col just have the date (without time)
    df[date_col] = df[date_col].dt.date
    #Merge all column names as a string
    id_col = ','.join(id_cols)
    #And as a column
    df[id_col] = df[id_cols].apply(lambda x:','.join(x.dropna().astype(str)),axis=1)
    #Drop any "id_col"-set that has a lower frequency than the max of the "shifts-list"
    freq = df[id_col].value_counts().to_frame()
    omit_idx = freq[freq[id_col]<=max(shifts)].index.to_list()
    if len(omit_idx)>0:
        df = df[~df[id_col].isin(omit_idx)].copy()
    #Change data structure to build the "shifting"
    df = df.pivot_table(index=[id_col,date_col],
                        **pivot_args,
                        fill_value=0)
    #Concatenate multiple columns if they are
    df.columns = ['_'.join([x for x in col]) if 
                  not isinstance(df.columns[0],str) #First element is not a string
                  else col for col in df.columns]
    #Bring the id_col for taking the set (unique values) in the next loop
    df = df.reset_index()
    #Each shift must be calculated at "id_col" level
    total = pd.DataFrame()
    for row in set(df[id_col]):
        #Set the id_col as index (again) to call all the rows with that id_col
        df_id = df.set_index(id_col).loc[row,:]
        #All possible dates from the min to the max of the subset
        tot_dates = pd.DataFrame(pd.date_range(start=df_id[date_col].min(), 
                                               end=df_id[date_col].max()).date, 
                                 columns=[date_col])
        df_id = df_id.merge(tot_dates,on=date_col,how='right').fillna(0)
        cols = df_id.columns[1:]
        #Start the "shifting"
        aux = df_id.copy()
        for i in shifts:
            aux = aux.join(df_id.iloc[:,1:].shift(i).rename(columns={x:f'{x}_{str(i).zfill(2)}' 
                                                                     for x in cols}))
        aux[id_col] = row
        total = total.append(aux,ignore_index=True)
    return total.set_index(id_cols+[date_col])

## Transform

In [ ]:
df,qbins = transf(df,['Retiro','Arribo'])
df.sample()

### Ridetime

In [ ]:
df['Ridetime'].hist()

Omit outliers with the IQR method

<https://online.stat.psu.edu/stat200/lesson/3/3.2>

In [ ]:
#Save the original length
dim_before = len(df)

df = iqr(df,'Ridetime')
df['Ridetime'].hist()

### Distance

In [ ]:
df['Distance'].hist()

In [ ]:
df = iqr(df,'Distance')
df['Distance'].hist()

### Velocity

In [ ]:
df['Velocity'].hist()

In [ ]:
df = iqr(df,'Velocity')
df['Velocity'].hist()

### Age

In [ ]:
df['Edad_Usuario'].hist()

In [ ]:
df = iqr(df,'Edad_Usuario')
df['Edad_Usuario'].hist()

In [ ]:
#Does the outlier cleaning affect the data length?
print(len(df)/dim_before)

### Demand

In [ ]:
'''!pip install seaborn'''
import seaborn as sns

color = sns.light_palette('LightSkyBlue', as_cmap=True)

#This is how the demand (and their avg age) looks through the daytime vs day of the week
df.pivot_table(index='HRmin_Retiro_cut',
               columns='DoW_Retiro',
               values='Edad_Usuario',
               aggfunc=['count','mean']).style.background_gradient(cmap=color).format('{:.1f}')
#But surely the demand does not distribute exactly the same along all stations

In [ ]:
demand = df.pivot_table(index=['Ciclo_Estacion_Retiro','HRmin_Retiro_cut'],
                    columns='DoW_Retiro',
                    values='Edad_Usuario',
                    aggfunc='count',
                    fill_value=0).reset_index()
demand.sample()

In [ ]:
demand.pivot_table(index='HRmin_Retiro_cut',
                   values=demand.columns[-7:],
                   aggfunc=['median','mean',perc70]
                  ).T.style.background_gradient(cmap=color).format('{:.1f}')

## TAD

In [ ]:
import seaborn as sns

color = sns.dark_palette("#69d", as_cmap=True)

#Example of what this function does
multishift(data=df.head(9).append(df.tail(9)),
           id_cols=['Ciclo_Estacion_Retiro'],
           date_col='Fecha_Retiro',
           shifts=[1,3,4],
           values='Bici',aggfunc='count',
           #columns=['HRmin_Retiro_cut']
           #columns=['HRmin_Retiro_cut','HRmin_Arribo_cut'],
           #values=['Bici','Genero_Usuario'],
           #aggfunc={'Bici':['sum','mean'],'Genero_Usuario':'count'}
          ).style.background_gradient(cmap=color).format('{:.1f}')

df = multishift(data=df,
                id_cols=['Ciclo_Estacion_Retiro'],
                date_col='Fecha_Retiro',
                shifts=range(1,10),
                columns='HRmin_Retiro_cut',
                values='Bici',
                aggfunc='count')
df.reset_index().to_csv(os.path.join(BASE_DIR,'bicis_shift.csv'),index=False)

In [38]:
#It took about 13min to complete, so we have now a backup file to not repeat that step
df = pd.read_csv(os.path.join(BASE_DIR,'bicis_shift.csv'),index_col=[0,1]).dropna()
df.sample()

,,"(-0.001, 8.595]","(10.455, 13.647]","(13.647, 15.841]","(15.841, 18.078]","(18.078, 19.535]","(19.535, 24.0]","(8.595, 10.455]","(-0.001, 8.595]_01","(10.455, 13.647]_01","(13.647, 15.841]_01",...,"(18.078, 19.535]_08","(19.535, 24.0]_08","(8.595, 10.455]_08","(-0.001, 8.595]_09","(10.455, 13.647]_09","(13.647, 15.841]_09","(15.841, 18.078]_09","(18.078, 19.535]_09","(19.535, 24.0]_09","(8.595, 10.455]_09"
Ciclo_Estacion_Retiro,Fecha_Retiro,,,,,,,,,,,,,,,,,,,,,
124,2019-05-25,1.000,10.000,2.000,7.000,4.000,2.000,7.000,24.000,31.000,14.000,...,5.000,9.000,14.000,19.000,20.000,30.000,7.000,6.000,3.000,24.000


In [39]:
#Save the columns with the previous dates info
prev = df.head().filter(regex='_\d+').columns.tolist()
actual = [x for x in df.columns if x not in prev]
print(actual)

['(-0.001, 8.595]', '(10.455, 13.647]', '(13.647, 15.841]', '(15.841, 18.078]', '(18.078, 19.535]', '(19.535, 24.0]', '(8.595, 10.455]']


## Daily Model

### Preprocessing

In [40]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X = df[prev].copy()
y = df[actual].sum(axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    train_size=0.77,
                                                    random_state=22)
mm_x = MinMaxScaler()

### Arquitecture

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression()

model_reg = Pipeline(steps=[('scaler', mm_x),
                            ('model', linear_reg)])

print('Score: ',model_reg.fit(X_train,y_train).score(X_test,y_test))
#The most relevant features to the model_reg
coef = pd.DataFrame(zip(X.columns,model_reg[1].coef_)).sort_values(1,0,0).reset_index(drop=True)
coef.head().append(coef.tail())

Score:  0.8967643672814342


,0,1
0,"(-0.001, 8.595]_07",160.352
1,"(8.595, 10.455]_07",93.151
2,"(13.647, 15.841]_07",55.503
3,"(15.841, 18.078]_07",50.518
4,"(-0.001, 8.595]_01",50.162
58,"(19.535, 24.0]_03",-13.364
59,"(19.535, 24.0]_08",-18.416
60,"(8.595, 10.455]_02",-21.493
61,"(-0.001, 8.595]_08",-25.503
62,"(-0.001, 8.595]_05",-48.251


In [42]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_jobs=-1,random_state=22)

model_forest = Pipeline(steps=[('scaler', mm_x),
                               ('model', forest)])

print(model_forest.fit(X_train,y_train).score(X_test,y_test))

#The most relevant features to the model_forest
pd.DataFrame(zip(X.columns,model_forest[1].feature_importances_)).sort_values(1,0,0).head(10).reset_index(drop=True)

0.9066158369976474


,0,1
0,"(13.647, 15.841]_07",0.317
1,"(19.535, 24.0]_07",0.117
2,"(-0.001, 8.595]_07",0.116
3,"(8.595, 10.455]_07",0.103
4,"(15.841, 18.078]_07",0.074
5,"(18.078, 19.535]_07",0.045
6,"(10.455, 13.647]_07",0.020
7,"(8.595, 10.455]_01",0.016
8,"(15.841, 18.078]_06",0.009
9,"(19.535, 24.0]_01",0.009


'''pip install xgboost'''
from xgboost.sklearn import XGBRegressor
xgb = XGBRegressor()

param_xgb = {'learning_rate':[x/100 for x in range(1,52,5)],
             'n_estimators':[x for x in range(100,1100,100)],
             'max_depth':[x for x in range(1,11)], 
             'min_child_weight':[x/10 for x in range(1,101,10)],
             'reg_lambda': [x/10 for x in range(1,101,10)]
            }
            
from sklearn.model_selection import RandomizedSearchCV
search_xgb = RandomizedSearchCV(param_distributions = param_xgb, 
                                cv = 4, 
                                n_jobs = -1, 
                                scoring = 'r2', 
                                estimator = xgb, 
                                verbose = 1,
                                n_iter = 70,
                                random_state = 22)

model_xgb = Pipeline(steps=[('scaler', mm_x),
                            ('model', search_xgb)])

print(model_xgb.fit(X_train,y_train).score(X_test,y_test))

#The most relevant features to the model_xgb
pd.DataFrame(zip(X.columns,model_xgb[1].best_estimator_.feature_importances_)).sort_values(1,0,0).head(10).reset_index(drop=True)

### Save the model

import pickle
with open(os.path.join(BASE_DIR,'bici_day_model.pkl'), 'wb') as f:
    pickle.dump((model_xgb,[prev,actual]), f)

import pickle    
with open(os.path.join(BASE_DIR,'bici_day_model.pkl'), 'rb') as f:
    modelo = pickle.load(f)

## Time-Window Model

### Preprocessing

In [45]:
df[actual].sum()

(-0.001, 8.595]    75725.000
(10.455, 13.647]   73988.000
(13.647, 15.841]   75311.000
(15.841, 18.078]   74704.000
(18.078, 19.535]   74717.000
(19.535, 24.0]     72202.000
(8.595, 10.455]    75630.000
dtype: float64

In [46]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X = df[prev].join(pd.DataFrame(model_forest.predict(X),index=X.index,columns=['day']))
y = df[actual[2]].values

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    train_size=0.77,
                                                    random_state=22)
mm_x = MinMaxScaler()

### Arquitecture

In [47]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression()

model_reg = Pipeline(steps=[('scaler', mm_x),
                            ('model', linear_reg)])

print('Score: ',model_reg.fit(X_train,y_train).score(X_test,y_test))
#The most relevant features to the model_reg
coef = pd.DataFrame(zip(X.columns,model_reg[1].coef_)).sort_values(1,0,0).reset_index(drop=True)
coef.head().append(coef.tail())

Score:  0.7425341431926671


,0,1
0,day,54.335
1,"(13.647, 15.841]_07",14.038
2,"(-0.001, 8.595]_05",6.212
3,"(13.647, 15.841]_02",3.764
4,"(13.647, 15.841]_01",3.318
59,"(15.841, 18.078]_05",-5.214
60,"(-0.001, 8.595]_01",-8.138
61,"(8.595, 10.455]_07",-8.985
62,"(-0.001, 8.595]_06",-9.387
63,"(-0.001, 8.595]_07",-20.015


In [48]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_jobs=-1,random_state=22)

model_forest = Pipeline(steps=[('scaler', mm_x),
                               ('model', forest)])

print(model_forest.fit(X_train,y_train).score(X_test,y_test))

#The most relevant features to the model_forest
pd.DataFrame(zip(X.columns,model_forest[1].feature_importances_)).sort_values(1,0,0).head(10).reset_index(drop=True)

0.7454001128717749


,0,1
0,day,0.574
1,"(13.647, 15.841]_07",0.143
2,"(13.647, 15.841]_01",0.024
3,"(15.841, 18.078]_02",0.009
4,"(13.647, 15.841]_06",0.007
5,"(15.841, 18.078]_04",0.007
6,"(-0.001, 8.595]_06",0.007
7,"(18.078, 19.535]_01",0.007
8,"(13.647, 15.841]_08",0.007
9,"(15.841, 18.078]_03",0.007


In [28]:
from xgboost.sklearn import XGBRegressor
xgb = XGBRegressor()

param_xgb = {'learning_rate':[x/100 for x in range(1,52,5)],
             'n_estimators':[x for x in range(100,1100,100)],
             'max_depth':[x for x in range(1,11)], 
             'min_child_weight':[x/10 for x in range(1,101,10)],
             'reg_lambda': [x/10 for x in range(1,101,10)]
            }
            
from sklearn.model_selection import RandomizedSearchCV
search_xgb = RandomizedSearchCV(param_distributions = param_xgb, 
                                cv = 4, 
                                n_jobs = -1, 
                                scoring = 'r2', 
                                estimator = xgb, 
                                verbose = 1,
                                n_iter = 70,
                                random_state = 22)

model_xgb = Pipeline(steps=[('scaler', mm_x),
                            ('model', search_xgb)])

print(model_xgb.fit(X_train,y_train).score(X_test,y_test))

#The most relevant features to the model_xgb
pd.DataFrame(zip(X.columns,model_xgb[1].best_estimator_.feature_importances_)).sort_values(1,0,0).head(10).reset_index(drop=True)

Fitting 4 folds for each of 70 candidates, totalling 280 fits
0.9118657187871996


,0,1
0,"(13.647, 15.841]_07",0.189
1,"(19.535, 24.0]_07",0.112
2,"(15.841, 18.078]_07",0.106
3,"(8.595, 10.455]_07",0.093
4,"(10.455, 13.647]_07",0.082
5,"(-0.001, 8.595]_07",0.072
6,"(18.078, 19.535]_07",0.046
7,"(19.535, 24.0]_01",0.025
8,"(8.595, 10.455]_01",0.022
9,"(10.455, 13.647]_06",0.022


### Save the model

In [30]:
import pickle
with open(os.path.join(BASE_DIR,'bici_day_model.pkl'), 'wb') as f:
    pickle.dump((model_xgb,[prev,actual]), f)

import pickle    
with open(os.path.join(BASE_DIR,'bici_day_model.pkl'), 'rb') as f:
    modelo = pickle.load(f)

## End

In [ ]:
time_exp(time.time() - start)
tone()